In [24]:
# 1) Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import opendatasets as od 
from tensorflow import keras
from keras.preprocessing.image import load_img,img_to_array
print(tf.__version__)


2.9.1


In [15]:
# 2) Download the datset
dataset_url = "https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition?select=test"
od.download(dataset_url)

Skipping, found downloaded files in "./fruit-and-vegetable-image-recognition" (use force=True to force download)


In [16]:
# 3) Create a list with the filepaths for training and testing
train_dir = Path('./fruit-and-vegetable-image-recognition/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path('./fruit-and-vegetable-image-recognition/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path('./fruit-and-vegetable-image-recognition/validation')
val_filepaths = list(test_dir.glob(r'**/*.jpg'))

In [17]:
def image_processing(filepath):
    # 4) Create a DataFrame with the filepath and the labels of the pictures

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

In [18]:
train_df = image_processing(train_filepaths)
test_df = image_processing(test_filepaths)
val_df = image_processing(val_filepaths)

In [19]:
print('-- Training set --\n')
print(f'Number of pictures: {train_df.shape[0]}\n')
print(f'Number of different labels: {len(train_df.Label.unique())}\n')
print(f'Labels: {train_df.Label.unique()}')

-- Training set --

Number of pictures: 2780

Number of different labels: 36

Labels: ['sweetpotato' 'turnip' 'spinach' 'pear' 'pomegranate' 'pineapple'
 'sweetcorn' 'banana' 'soy beans' 'carrot' 'orange' 'paprika' 'raddish'
 'ginger' 'grapes' 'bell pepper' 'garlic' 'lettuce' 'potato'
 'chilli pepper' 'cauliflower' 'beetroot' 'cucumber' 'onion' 'apple'
 'lemon' 'cabbage' 'corn' 'watermelon' 'mango' 'peas' 'kiwi' 'eggplant'
 'tomato' 'jalepeno' 'capsicum']


In [20]:
train_df.head(5)

,Filepath,Label
0,fruit-and-vegetable-image-recognition/train/sw...,sweetpotato
1,fruit-and-vegetable-image-recognition/train/tu...,turnip
2,fruit-and-vegetable-image-recognition/train/sp...,spinach
3,fruit-and-vegetable-image-recognition/train/sp...,spinach
4,fruit-and-vegetable-image-recognition/train/tu...,turnip


In [25]:
# Create a DataFrame with one Label of each category
df_unique = train_df.copy().drop_duplicates(subset=["Label"]).reset_index()

# Display some pictures of the dataset
fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(8, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_unique.Filepath[i]))
    ax.set_title(df_unique.Label[i], fontsize = 12)
plt.tight_layout(pad=0.5)
plt.show()

AttributeError: module 'PIL.Image' has no attribute 'Resampling'

AttributeError: module 'PIL.Image' has no attribute 'Resampling'

<Figure size 576x504 with 36 Axes>

In [26]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [27]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

Found 2780 validated image filenames belonging to 36 classes.


In [30]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False 